In [2]:
import spacy
nlp = spacy.load('en_core_web_sm')
text="West Chesterre was references in Mr. Deeds."
doc=nlp(text)

In [3]:
#There are two differerent approaches to NLP with spacy.
#1.Rules based approach :- This refers to providing rules to process words in textv
#2.Machine learning approach:- This refers to usiing machine learning to process words in text

#We need to use one of these approaches to prepare the model to work on text for a certain domain in which the off the shelf model fails.



In [4]:
#Rules Based Approach
#EntityRuler is a grammetical rules thay can be used to assign entity labels to tokens

#The entity ruler is a list of disctionary which specifies which label to assign to a apecific psttern i.e token.
ruler = nlp.add_pipe("entity_ruler")

patterns=[
    {"label":"GRE","pattern":"West Chesterre"},
]
ruler.add_patterns(patterns)
doc=nlp(text)

for ent in doc.ents:
    print(ent.text,ent.label_)

#This will still print West Chesterre as label other than GRE.
#It i because the pipeline is sequential since the entity ruler was added at last it will work on the data after the NER so the labe is assigned before entity ruler can act o it

nlp.analyze_pipes()



West Chesterre NORP
Deeds PERSON


{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'entity_ruler': {'assigns': ['doc.ents', 'token.ent_type', 'token.ent_iob'],
   'requires': [],
   'scores': ['ents_f', 'ent

In [5]:
#To make sure that entity ruler act on data before the NER we should apply it to pipeline before NER
nlp1 = spacy.load('en_core_web_sm')

text = "Apple is looking at buying U.K. startup for $1 billion"

ruler1=nlp1.add_pipe("entity_ruler",before='ner')

patterns=[
    {'label':'TCOM','pattern':'Apple'}
]
ruler1.add_patterns(patterns)

doc = nlp1(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

print(nlp1.pipe_names)

#With the hep of list of disctionary we can add multiple patterns to make the model custom to our specific dataset.

#This entity ruler is one of the ways to solve Toponym resolution problem i.e. the same named entity that can have different labels depending upon the context.capitalize
 


Apple TCOM
U.K. GPE
$1 billion MONEY
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'entity_ruler', 'ner']


In [6]:
#The matcher is used to apply pattern in data which is not necessary to add as a entity but as a certain structure in data.
#We use matcher to implement a pattern which is not necessarily dependent upon the pattern of speech.

from spacy.matcher import Matcher
nlp2= spacy.load('en_core_web_sm')
matcher= Matcher(nlp2.vocab)
pattern=[{"LIKE_EMAIL":True},{"POS":"PROPN"}] #This is one of the matcher in the spacy. It takes the content of the text to find the Email like text
matcher.add("EMAIL_ADDRESS",[pattern])

doc2=nlp2("I like to send email to abc@gmail.com Apple")

matches=matcher(doc2)
print(matches) #This prints a list of tuple in which first item corresponds to the matched pattern name and the furthur 2 gives the location of the matched token

print(nlp2.vocab[matches[0][0]].text)#We can extract which pattern is matched like this
print(doc2[matches[0][1]:matches[0][2]])#We can get the matched token like this

#There are many parameters that we can add to the matcher to make it more customized for our domain of text.

[(16571425990740197027, 6, 8)]
EMAIL_ADDRESS
abc@gmail.com Apple


In [7]:
#We can make complex matches like selecting all the instances of a verb followed by noun.
#This is a more powerful feature of spacy which are not present in other inferior libraries.
with open("wiki_mlk.txt","r") as f:
    text=f.read()

print(text)
#Lets extract all the proper nouns in this text.

nlp3=spacy.load("en_core_web_md")
matcher=Matcher(nlp3.vocab)
pattern=[{"POS":"PROPN"}] #This extracts any occurance of part of speech where the word is a proper noun.
matcher.add("PROPER_NOUN",[pattern])
doc3=nlp3(text)
matches=matcher(doc3)

for match in matches:
    print(match,doc3[match[1]:match[2]])
#This can extract all the single word proper nouns but it is limited
#As if the whole name of a person should be extract  as single but here it takes each individually


Martin Luther King Jr. (born Michael King Jr.; January 15, 1929 – April 4, 1968) was an American Baptist minister and activist who became the most visible spokesman and leader in the American civil rights movement from 1955 until his assassination in 1968. King advanced civil rights through nonviolence and civil disobedience, inspired by his Christian beliefs and the nonviolent activism of Mahatma Gandhi. He was the son of early civil rights activist and minister Martin Luther King Sr.

King participated in and led marches for blacks' right to vote, desegregation, labor rights, and other basic civil rights.[1] King led the 1955 Montgomery bus boycott and later became the first president of the Southern Christian Leadership Conference (SCLC). As president of the SCLC, he led the unsuccessful Albany Movement in Albany, Georgia, and helped organize some of the nonviolent 1963 protests in Birmingham, Alabama. King helped organize the 1963 March on Washington, where he delivered his famous 

In [8]:
#To rectify the error of the previous notebook, I have to change the code to the following
nlp3=spacy.load("en_core_web_md")
matcher=Matcher(nlp3.vocab)
pattern=[{"POS":"PROPN","OP":"+"}]#This op detrmines how often to make the match to token pattern
#Here the '+' means to match with token one or more times
#What this means is it can match the single token as well as with multiple tokens which are compounded

matcher.add("PROPER_NOUN",[pattern],greedy='LONGEST') 
#THis takes matches single as well as compound and in case of compound it will take the longest one

doc3=nlp3(text)
matches=matcher(doc3)

matches.sort(key=lambda x:x[1]) #This is to sort as the matches according to beginning occurance of the longest token

for match in matches:
    print(match,doc3[match[1]:match[2]])

(451313080118390996, 0, 4) Martin Luther King Jr.
(451313080118390996, 6, 9) Michael King Jr.
(451313080118390996, 10, 11) January
(451313080118390996, 15, 16) April
(451313080118390996, 23, 24) Baptist
(451313080118390996, 49, 50) King
(451313080118390996, 69, 71) Mahatma Gandhi
(451313080118390996, 83, 88) Martin Luther King Sr.
(451313080118390996, 89, 90) King
(451313080118390996, 113, 114) King
(451313080118390996, 117, 118) Montgomery
(451313080118390996, 128, 132) Southern Christian Leadership Conference
(451313080118390996, 133, 134) SCLC
(451313080118390996, 140, 141) SCLC
(451313080118390996, 146, 148) Albany Movement
(451313080118390996, 149, 150) Albany
(451313080118390996, 151, 152) Georgia
(451313080118390996, 163, 164) Birmingham
(451313080118390996, 165, 166) Alabama
(451313080118390996, 167, 168) King
(451313080118390996, 172, 173) March
(451313080118390996, 174, 175) Washington
(451313080118390996, 193, 195) Lincoln Memorial
(451313080118390996, 198, 199) SCLC
(451313

In [9]:
#Now lets make it more complicated lets find matches where a proper noun is followed by a verb
nlp3=spacy.load("en_core_web_md")
matcher=Matcher(nlp3.vocab)
pattern=[{"POS":"PROPN","OP":"+"},{"POS":"VERB"}]
matcher.add("PROPER_NOUN",[pattern],greedy='LONGEST') 

doc3=nlp3(text)
matches=matcher(doc3)

matches.sort(key=lambda x:x[1]) #This is to sort as the matches according to beginning occurance of the longest token

for match in matches:
    print(match,doc3[match[1]:match[2]])

(451313080118390996, 49, 51) King advanced
(451313080118390996, 89, 91) King participated
(451313080118390996, 113, 115) King led
(451313080118390996, 198, 200) SCLC put
(451313080118390996, 247, 252) Director J. Edgar Hoover considered
(451313080118390996, 322, 324) King won
(451313080118390996, 485, 488) United States beginning


In [10]:
#Now lets do a more complex tash, we will take a paragraph and find all the instances where a certain quote is given by some one
import json
nlp4=spacy.load("en_core_web_md")
matcher4=Matcher(nlp3.vocab)
with open ("alice.json", "r") as f:
    data = json.load(f)

text = data[0][2][0]
print (text)

text.replace("`", "'") #This is somehow not working
print (text)

speak_lemmas = ["think", "say"]

matcher = Matcher(nlp.vocab)
pattern = [{'ORTH': "`"}, #So the sequence must start with '
           {'IS_ALPHA': True, "OP": "+"},#Then followed by one or more alphabetic tokens
            {'IS_PUNCT': True, "OP": "*"}, #Could possibly be a punctuation mark like comma
            {'ORTH': "'"},#Terminated by '
            {'POS':'VERB',"LEMMA":{"IN":speak_lemmas}}, #This checks if the quote is followed by a certain version of verb Speak
            {'POS':'PROPN', 'OP': '+'},
            ]
#We can also add more than one pattern as there can be multiple slightly different variant of the equired pattern.
matcher.add("PROPER_NOUNS", [pattern], greedy='LONGEST')
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'
Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'
1
(3232560085755078826, 47, 60) `and what is the use of a book,' thought Alice
